In [1]:
%%bash
pip install symforce

In [2]:
import symforce.symbolic as sym
import numpy as np


In [6]:
pose =sym.Pose2(
    t=sym.V2.symbolic('t'),
    R=sym.Rot2.symbolic('R')
)
landmark= sym.V2.symbolic('L')

In [8]:
landmark_body=pose.inverse() * landmark     

In [10]:
landmark_body.jacobian(pose)

[-L0*R_im + L1*R_re + t0*R_im - t1*R_re, -R_re, -R_im]
[-L0*R_re - L1*R_im + t0*R_re + t1*R_im, R_im, -R_re]